# **1. Import Library**

In [23]:
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, f_classif

# **2. Memuat Dataset dari Hasil Clustering**

In [9]:
from os import read
from google.colab import drive
drive.mount('/content/drive')

file_path = "/content/drive/My Drive/dataset_ml/finally_hasil_clustering.csv"
df = pd.read_csv(file_path)
df.info()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45360 entries, 0 to 45359
Data columns (total 47 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   upah                           45360 non-null  int64  
 1   ump                            45360 non-null  float64
 2   gk                             45360 non-null  float64
 3   peng                           45360 non-null  float64
 4   Provinsi_BALI                  45360 non-null  float64
 5   Provinsi_BANTEN                45360 non-null  float64
 6   Provinsi_BENGKULU              45360 non-null  float64
 7   Provinsi_DI YOGYAKARTA         45360 non-null  float64
 8   Provinsi_DKI JAKARTA           45360 non-null  float64
 9   Provinsi_GORONTALO             45360 non-null  float64
 10  Provinsi_INDONESIA             45360 

# **3. Data Splitting**

In [6]:
X = df.drop(columns=['Cluster', 'Provinsi', 'jenis_x', 'daerah_x']) # menghapus kolom numerik
y = df['Cluster']

print("Dimensi Fitur (X):", X.shape)
print("Dimensi Targe (y):", y.shape)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)
print("Jumlah Data latih:", X_train.shape[0])
print("Jumlah Data latih:", X_test.shape[0])

Dimensi Fitur (X): (45360, 43)
Dimensi Targe (y): (45360,)
Jumlah Data latih: 36288
Jumlah Data latih: 9072


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

In [29]:
# model random forest
model_rf = RandomForestClassifier(random_state=42, n_estimators=100)
model_rf.fit(X_train, y_train)

# model Knn
model_knn = KNeighborsClassifier(n_neighbors=5)
model_knn.fit(X_train, y_train)

print("semua moodel telah dilatih")

semua moodel telah dilatih


- Algoritma Random Forest adalah pengembangan dari Decision Tree, yang terdiri dari banyak pohon keputusan (ensemble learning).
Setiap pohon dilatih dengan subset data yang berbeda, dan keputusan akhir diambil berdasarkan voting mayoritas dari seluruh pohon.
Model ini lebih robust dan mengurangi overfitting, dibandingkan dengan Decision Tree tunggal.
Parameter n_estimators=100 menunjukkan jumlah pohon yang digunakan dalam ensemble.

- Algoritma KNN bekerja dengan mencari k tetangga terdekat untuk menentukan kelas dari suatu sampel.
Algoritma ini tidak mempelajari pola selama training, tetapi menghitung jarak antar data saat prediksi dilakukan (lazy learning).
Parameter n_neighbors=5 berarti model akan mempertimbangkan 5 tetangga terdekat untuk menentukan kelas suatu data.

## **b. Evaluasi Model Klasifikasi**

In [30]:
y_pred_rf = model_rf.predict(X_test)
y_pred_knn = model_knn.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
accuracy_knn = accuracy_score(y_test, y_pred_knn)

print(f" Classification report (Random Forest):")
print (classification_report(y_test, y_pred_rf))

print(f" Classification report (KNN):")
print (classification_report(y_test, y_pred_knn))




 Classification report (Random Forest):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2009
           1       1.00      1.00      1.00      1069
           2       1.00      1.00      1.00      2624
           3       1.00      1.00      1.00      3370

    accuracy                           1.00      9072
   macro avg       1.00      1.00      1.00      9072
weighted avg       1.00      1.00      1.00      9072

 Classification report (KNN):
              precision    recall  f1-score   support

           0       0.86      0.87      0.86      2009
           1       0.91      0.89      0.90      1069
           2       0.93      0.91      0.92      2624
           3       0.88      0.89      0.88      3370

    accuracy                           0.89      9072
   macro avg       0.89      0.89      0.89      9072
weighted avg       0.89      0.89      0.89      9072



1. Random Forest Classifier
- **Akurasi:** 100% (1.00000)
- **Precision, Recall, F1-score:** 100% untuk semua kelas
- **Kelebihan:**Lebih stabil karena menggunakan ensemble learning (menggabungkan banyak pohon keputusan).
- **Kekurangan:**Masih mengalami overfitting, seperti yang terlihat dari akurasi sempurna.

2. K-Nearest Neighbors (KNN)
- **Akurasi:** 89% (0.88955)
- **Precision, Recall, F1-score:** Rata-rata 89%
- **Kelebihan:**Tidak mengalami overfitting seperti  Random Forest.Lebih fleksibel dalam menangani berbagai pola data.
- **Kekurangan:**Lebih lambat dibandingkan Random Forest, terutama pada dataset besar karena harus menghitung jarak ke semua sampel saat prediksi.
Kinerja sangat bergantung pada pemilihan parameter jumlah tetangga (k).



## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [31]:
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [ 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}
grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_rf.fit(X_train, y_train)


param_grid_knn = {
    'n_neighbors': [3, 5, 7, 10],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}
grid_knn = GridSearchCV(KNeighborsClassifier(), param_grid_knn, cv=5, scoring='accuracy', n_jobs=-1)
grid_knn.fit(X_train, y_train)


print("parameter terbaik unutk model random forest:", grid_rf.best_params_)
print("parameter terbaik untuk model knn:", grid_knn.best_params_)



parameter terbaik unutk model random forest: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
parameter terbaik untuk model knn: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

In [32]:
best_rf = grid_rf.best_estimator_
best_knn = grid_knn.best_estimator_

y_pred_rf_best = best_rf.predict(X_test)
y_pred_knn_best = best_knn.predict(X_test)

accuracy_rf_best = accuracy_score(y_test, y_pred_rf_best)
accuracy_knn_best = accuracy_score(y_test, y_pred_knn_best)

print(f"Hasil Evaluasi setelah tuning")
print(f" Random Forest: {accuracy_rf_best:.5f} | knn: {accuracy_knn_best:.5f}\n")

for model_name, y_pred in zip(["Random Forest", "KNN"],
                              [y_pred_rf_best, y_pred_knn_best]):
  print (f"Classification report ({model_name} - Tuned):\n{classification_report(y_test, y_pred)}\n")

Hasil Evaluasi setelah tuning
 Random Forest: 1.00000 | knn: 0.98280

Classification report (Random Forest - Tuned):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2009
           1       1.00      1.00      1.00      1069
           2       1.00      1.00      1.00      2624
           3       1.00      1.00      1.00      3370

    accuracy                           1.00      9072
   macro avg       1.00      1.00      1.00      9072
weighted avg       1.00      1.00      1.00      9072


Classification report (KNN - Tuned):
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      2009
           1       0.99      0.99      0.99      1069
           2       0.99      0.99      0.99      2624
           3       0.98      0.98      0.98      3370

    accuracy                           0.98      9072
   macro avg       0.98      0.98      0.98      9072
weighted avg       0.98      0

In [33]:
# Pilih fitur terbaik berdasarkan skor ANOVA F-test
selector = SelectKBest(f_classif, k=5)
X_train_new = selector.fit_transform(X_train, y_train)
X_test_new = selector.transform(X_test)

print("fitur terpilih:", X.columns[selector.get_support()])

fitur terpilih: Index(['upah', 'ump', 'Provinsi_DKI JAKARTA', 'Provinsi_PAPUA', 'Upah_Binned'], dtype='object')


In [37]:
best_rf = RandomForestClassifier(n_estimators=50, max_depth=5, min_samples_leaf=5, random_state=42)
best_rf.fit(X_train, y_train)

best_knn = KNeighborsClassifier(n_neighbors=3, weights='uniform', metric='manhattan')
best_knn.fit(X_train, y_train)

cv_score_rf = cross_val_score(best_rf, X_train, y_train, cv=5, scoring='accuracy')
cv_score_knn = cross_val_score(best_knn, X_train, y_train, cv=5, scoring='accuracy')

print(f"Cross-Validation Accuracy (Random Forest - Tuned): {cv_score_rf.mean():.4f}")
print(f"Cross-Validation Accuracy (KNN - Tuned): {cv_score_knn.mean():.4f}")

Cross-Validation Accuracy (Random Forest - Tuned): 0.9406
Cross-Validation Accuracy (KNN - Tuned): 0.9529


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

1. **Perbandingan Hasil Evaluasi sebelum dan sesudah setelah tuning**

##**Model: Random Forest**
- Akurasi Sebelum Tuning: 100%
- Akurasi Setelah Tuning: 94.06%
- Random Forest mengalami penurunan akurasi dari 100% ke 94.06%, yang berarti model menjadi lebih generalizable dan tidak terlalu overfitting terhadap data pelatihan.
- Kelemahan:
 Performa lebih lambat dibandingkan Decision Tree karena menggunakan banyak pohon dalam prediksi.
 Meskipun lebih baik dalam mengurangi overfitting, model ini masih memiliki potensi peningkatan akurasi.

**Rekomendasi Tindakan lanjutan**
- Tambahkan lebih banyak n_estimators untuk meningkatkan stabilitas dan akurasi model.
- Eksperimen dengan metode boosting seperti AdaBoost atau Gradient Boosting untuk hasil yang lebih baik.

##**Model: KNN**
- Akurasi Sebelum Tuning: 98.28%
- Akurasi Setelah Tuning: 95.29%
- Akurasi KNN mengalami sedikit penurunan dari 98.28% ke 95.29%, tetapi masih cukup tinggi. Ini menunjukkan bahwa model masih cukup baik dalam memprediksi data uji meskipun telah disesuaikan parameternya.
- kelemahan:Kinerja menurun pada dataset besar, karena KNN membutuhkan waktu lebih lama untuk menghitung jarak antar data saat prediksi.
Sensitif terhadap skala fitur, sehingga normalisasi sangat diperlukan.

**Rekomendasi Tindakan Lanjutan:**
- Eksperimen dengan jumlah tetangga (n_neighbors) yang lebih besar atau lebih kecil.
- Coba metrik jarak yang berbeda seperti manhattan atau cosine similarity.
Gunakan KD-Tree atau Ball-Tree untuk mempercepat perhitungan pada dataset besar.


